In [1]:
from datetime import datetime
import requests
import psycopg2 as ps2
import psycopg2.extras as extras 
import pandas as pd

url = f"http://api.exchangerate.host/timeframe"

response = requests.get(url, params={'access_key': "1731467954af1a70db43b92e62e2e131",
                                'source': "BTC",
                                'currencies': "RUB",
                                'start_date':"2023-10-01",
                                'end_date':"2023-10-31",
                                'format': 1})
data = response.json()


pg_hostname = 'localhost'
pg_port = '5432'
pg_username = 'postgres'
pg_pass = 'password'
pg_db = 'test'

conn = ps2.connect(host=pg_hostname,
                   port=pg_port,
                   user=pg_username,
                   password=pg_pass,
                   database=pg_db)
cursor = conn.cursor()
cursor.execute(f"""CREATE TABLE IF NOT EXISTS rates_month (
              id serial primary key,
              dates date,
              source varchar(255),
              currencies varchar(255),
              rate double precision);""")
conn.commit()

for date, quotes in data['quotes'].items():
    for currency, rate in quotes.items():
        insert_query = f"""
        INSERT INTO rates_month(dates, source, currencies, rate) VALUES
        ('{date}', '{data['source']}', '{currency}', {rate})
        """
        cursor.execute(insert_query)
conn.commit()

select_query = "SELECT dates, source, currencies, rate FROM rates_month"
cursor.execute(select_query)
conn.commit()
cursor.close()
conn.close()
conn = ps2.connect(host=pg_hostname,
                   port=pg_port,
                   user=pg_username,
                   password=pg_pass,
                   database=pg_db)

query = "SELECT dates, source, currencies, rate FROM rates_month"
df = pd.read_sql_query(query, conn)
conn.close()
#день с максимальным курсом
max_rate_day = df[df['rate'] == df['rate'].max()]['dates'].values[0]
#день с минимальным курсом
min_rate_day = df[df['rate'] == df['rate'].min()]['dates'].values[0]


# Максимальное значение курса
max_rate = df['rate'].max()


# Минимальное значение курса
min_rate = df['rate'].min()


# Среднее значение курса за весь месяц
avg_rate = df['rate'].mean()


# Значение курса на последний день месяца
last_day_rate = df.sort_values(by='dates', ascending=False)['rate'].values[0]

#месяц в котором проходит действо
month =df.sort_values(by='dates', ascending=True)['dates'].values[0].month
#замутим датасет из всего этого
one = pd.DataFrame({'MAX_rate_day': max_rate_day,'MIN_rate_day': min_rate_day,
                       'MAX_rate': max_rate,'MIN_rate': min_rate,
                       'AVG_rate': avg_rate,'LAST_DAY_rate': last_day_rate,
                       'Month': month,'Currencies': currency,},index=[0])
conn = ps2.connect(host=pg_hostname,
                   port=pg_port,
                   user=pg_username,
                   password=pg_pass,
                   database=pg_db)
cursor = conn.cursor()
cursor.execute(f"""CREATE TABLE IF NOT EXISTS one_raw_table (
              MAX_rate_day date,
              MIN_rate_day date,
              MAX_rate double precision,
              MIN_rate double precision,
              AVG_rate double precision,
              LAST_DAY_rate double precision,
              Month int8,
              currencies varchar(255));""")
conn.commit()


def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, ps2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    print("the dataframe is inserted") 
  

execute_values(conn, one, 'one_raw_table') 
conn.commit()
cursor.close()
conn.close()

#кто дочитал получает веселую хохму: я каким-то странным образом в запросе при манипуляциях со столбцом 'dates' из таблицы с данными за месяц
#  умудрился кириллицей букву 'e' напечатать -  и в итоге подсчет статистики у меня занял 6 часов)
#  пишу возьми из столбца 'dates' - а оно отвечает нетю такого) - набрел на идею что этот столбец индекс - еще час с этим боролся)
#  - а оказалась всего-то 'e' в запросе русская))) чудеса))


the dataframe is inserted


C:\Users\sever\AppData\Local\Temp\ipykernel_26056\1048890079.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
